In [1]:
import math
import numpy as np
''' If pygame is not installed already, enter: python -m pip install pygame in the anaconda powershell prompt'''
import pygame

import sys
import time
import copy
import random

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
X = "X"
O = "O"
EMPTY = None

In [3]:
# For trials

a = [[X, X, O],
    [O, EMPTY, X],
    [X, X, X]]

In [4]:
# for ease in calculating whose return it is, or if anyone has won

def truth_values(matrix, value):
    
    a =  np.zeros((len(matrix),len(matrix[0])), dtype=np.int16)
    
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] == value:
                a[i][j] = True
            else:
                a[i][j] = False
    return a
                

In [5]:
'''The function initial_state returns the starting state of the board. 
I have chosen to represent the board as a list of three lists (representing the three rows of the board), 
where each internal list contains three values that are either X, O, or EMPTY'''

def initial_state():
    """
    Returns starting state of the board.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]


In [6]:
'''The player function should take a board state as input, and return which player’s turn it is (either X or O).
In the initial game state, X gets the first move. Subsequently, the player alternates with each additional move.
Any return value is acceptable if a terminal board is provided as input (i.e., the game is already over).'''

def player(board):
    """
    Returns player who has the next turn on a board.
    """
    x_truth = truth_values(board,X)
    x_row = sum(x_truth)
    x_total = sum(x_row)
    
    o_truth = truth_values(board,O)
    o_row = sum(o_truth)
    o_total = sum(o_row)
    
    if x_total - o_total == 0:
        return X
    else:
        return O

In [7]:
'''The actions function should return a set of all of the possible actions that can be taken on a given board.
Each action should be represented as a tuple (i, j) where i corresponds to the row of the move (0, 1, or 2) 
and j corresponds to which cell in the row corresponds to the move (also 0, 1, or 2).
Possible moves are any cells on the board that do not already have an X or an O in them.
Any return value is acceptable if a terminal board is provided as input.'''

def actions(board):
    """
    Returns set of all possible actions (i, j) available on the board.
    """
    
    moves = []
    
    for i in range(3):
        for j in range(3):
            if board[i][j] == None:
                moves.append((i,j))
                
    return moves

In [8]:
'''The result function takes a board and an action as input, and should return a new board state, 
without modifying the original board. If action is not a valid action for the board, the program should raise an exception.
The returned board state should be the board that would result from taking the original input board, 
and letting the player whose turn it is make their move at the cell indicated by the input action.
Importantly, the original board should be left unmodified: since Minimax will ultimately require 
considering many different board states during its computation. 
This means that simply updating a cell in board itself is not a correct implementation of the result function. 
Make a deep copy of the board first before making any changes.'''

def result(board, action):
    """
    Returns the board that results from making move (i, j) on the board.
    """
    board = copy.deepcopy(board)
    
    if action not in actions(board):
        raise Exception("Sorry, the action is invalid")
    else:
        new = board
        new[action[0]][action[1]] = player(board)
        
    return new

In [9]:
'''The winner function should accept a board as input, and return the winner of the board if there is one.
If the X player has won the game, the function should return X. 
If the O player has won the game, the function should return O.
One can win the game with three of their moves in a row horizontally, vertically, or diagonally.
I assume that there will be at most one winner (that is, no board will ever have both players with three-in-a-row, 
since that would be an invalid board state).
If there is no winner of the game (either because the game is in progress, or because it ended in a tie), 
the function should return None.'''

def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2]:
            win = board[i][0]
            return win
        elif board[0][i] == board[1][i] == board[2][i]:
            win = board[0][i]
            return win
        elif board[0][0] == board[1][1] == board[2][2]:
            win = board[0][0]
            return win
        elif board[2][0] == board[1][1] == board[0][2]:
            win = board[2][0]
            return win
        else:
            win = None
            
    return win

In [10]:
'''The terminal function should accept a board as input, and return a boolean value indicating whether the game is over.
If the game is over, either because someone has won the game or because all cells have been filled without anyone winning, 
the function should return True.
Otherwise, the function should return False if the game is still in progress.'''

def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """
        
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] and board[i][0] is not None:
            win = True
            return win
        elif board[0][i] == board[1][i] == board[2][i] and board[0][i] is not None:
            win = True
            return win
        elif board[0][0] == board[1][1] == board[2][2] and board[0][0] is not None:
            win = True
            return win
        elif board[2][0] == board[1][1] == board[0][2] and board[2][0] is not None:
            win = True
            return win
        elif 1 not in truth_values(board, None):
            win = True
        else:
            win = False
        
            
    return win

In [11]:
'''The utility function should accept a terminal board as input and output the utility of the board.
If X has won the game, the utility is 1. If O has won the game, the utility is -1. If the game has ended in a tie, 
the utility is 0.
I assume utility will only be called on a board if terminal(board) is True.'''

def utility(board):
    """
    Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
    """
    if terminal(board):
        win = winner(board)
        if win == X:
            return 1
        elif win == O:
            return -1
        else:
            return 0

In [12]:
def max_value(boardCopy):
    if terminal(boardCopy):
        return utility(boardCopy)
    
    v = -math.inf
    current_actions = actions(boardCopy)
    
    for action in current_actions:
        v = max(v, min_value(result(boardCopy, action)))
    return v

In [13]:
def min_value(boardCopy):
    
    if terminal(boardCopy):
        return utility(boardCopy)
    
    v = math.inf
    current_actions = actions(boardCopy)
    
    for action in current_actions:
        v = min(v, max_value(result(boardCopy, action)))
    return v

In [14]:
'''The minimax function should take a board as input, and return the optimal move for the player to move on that board.
The move returned should be the optimal action (i, j) that is one of the allowable actions on the board. 
If multiple moves are equally optimal, any of those moves is acceptable.
If the board is a terminal board, the minimax function should return None.'''

def minimax(board):
    """
    Returns the optimal action for the current player on the board.
    """
    current_player = player(board)
    current_actions = actions(board)
    
    if terminal(board):
        return None
    
    if board == initial_state():
        i,j = random.randint(0,2), random.randint(0,2)
        return i, j
    
    if current_player == X:
        best_score = -math.inf
        best_move = None
        
        for action in current_actions:
            boardCopy = copy.deepcopy(board)
            new = result(boardCopy,action)
            score = min_value(new)
            if (score > best_score):
                best_score = score
                best_move = action
    else:
        best_score = math.inf
        best_move = None
        for action in current_actions:
            boardCopy = copy.deepcopy(board)
            new = result(boardCopy,action)
            score = max_value(new)
            if score < best_score:
                best_score = score
                best_move = action
    return best_move
  
    
    # TO DO
            

In [15]:
pygame.init()
size = width, height = 600, 400

# Colors
black = (0, 0, 0)
white = (255, 255, 255)

screen = pygame.display.set_mode(size)

mediumFont = pygame.font.Font("OpenSans-Regular.ttf", 28)
largeFont = pygame.font.Font("OpenSans-Regular.ttf", 40)
moveFont = pygame.font.Font("OpenSans-Regular.ttf", 60)

user = None
board = initial_state()
ai_turn = False

while True:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()

    screen.fill(black)

    # Let user choose a player.
    if user is None:

        # Draw title
        title = largeFont.render("Play Tic-Tac-Toe", True, white)
        titleRect = title.get_rect()
        titleRect.center = ((width / 2), 50)
        screen.blit(title, titleRect)

        # Draw buttons
        playXButton = pygame.Rect((width / 8), (height / 2), width / 4, 50)
        playX = mediumFont.render("Play as X", True, black)
        playXRect = playX.get_rect()
        playXRect.center = playXButton.center
        pygame.draw.rect(screen, white, playXButton)
        screen.blit(playX, playXRect)

        playOButton = pygame.Rect(5 * (width / 8), (height / 2), width / 4, 50)
        playO = mediumFont.render("Play as O", True, black)
        playORect = playO.get_rect()
        playORect.center = playOButton.center
        pygame.draw.rect(screen, white, playOButton)
        screen.blit(playO, playORect)

        # Check if button is clicked
        click, _, _ = pygame.mouse.get_pressed()
        if click == 1:
            mouse = pygame.mouse.get_pos()
            if playXButton.collidepoint(mouse):
                time.sleep(0.2)
                user = X
            elif playOButton.collidepoint(mouse):
                time.sleep(0.2)
                user = O

    else:

        # Draw game board
        tile_size = 80
        tile_origin = (width / 2 - (1.5 * tile_size),
                       height / 2 - (1.5 * tile_size))
        tiles = []
        for i in range(3):
            row = []
            for j in range(3):
                rect = pygame.Rect(
                    tile_origin[0] + j * tile_size,
                    tile_origin[1] + i * tile_size,
                    tile_size, tile_size
                )
                pygame.draw.rect(screen, white, rect, 3)

                if board[i][j] != EMPTY:
                    move = moveFont.render(board[i][j], True, white)
                    moveRect = move.get_rect()
                    moveRect.center = rect.center
                    screen.blit(move, moveRect)
                row.append(rect)
            tiles.append(row)

        game_over = terminal(board)  # Would be True if game is over, False if game running
        players = player(board) # X or O depending on the board

        # Show title
        if game_over is True:
            winners = winner(board)
            if winners is None:
                title = f"Game Over: Tie."
            else:
                title = f"Game Over: {winners} wins."
        elif user == players:
            title = f"Play as {user}"
        else:
            title = f"Computer thinking..."
        title = largeFont.render(title, True, white)
        titleRect = title.get_rect()
        titleRect.center = ((width / 2), 30)
        screen.blit(title, titleRect)

        # Check for AI move
        if user != players and not game_over:
            if ai_turn:
                time.sleep(0.5)
                move = minimax(board)
                board = result(board, move)
                ai_turn = False
            else:
                ai_turn = True

        # Check for a user move
        click, _, _ = pygame.mouse.get_pressed()
        if click == 1 and user == players and not game_over:
            mouse = pygame.mouse.get_pos()
            for i in range(3):
                for j in range(3):
                    if (board[i][j] == EMPTY and tiles[i][j].collidepoint(mouse)):
                        board = result(board, (i, j))

        if game_over:
            againButton = pygame.Rect(width / 3, height - 65, width / 3, 50)
            again = mediumFont.render("Play Again", True, black)
            againRect = again.get_rect()
            againRect.center = againButton.center
            pygame.draw.rect(screen, white, againButton)
            screen.blit(again, againRect)
            click, _, _ = pygame.mouse.get_pressed()
            if click == 1:
                mouse = pygame.mouse.get_pos()
                if againButton.collidepoint(mouse):
                    time.sleep(0.2)
                    user = None
                    board = initial_state()
                    ai_turn = False

    pygame.display.flip()

SystemExit: 

C:\Users\Sama\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
